### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 8, 14)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-08-14
yesterday: 2023-08-11 00:00:00


In [3]:
yesterday = yesterday.date()
week_ago = yesterday -timedelta(days = 8)
print(f'a week ago: {week_ago}')
print(f'yesterday: {yesterday}')

a week ago: 2023-08-03
yesterday: 2023-08-11


### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [4]:
cols = 'name price_w price_d percent perform mrkt '.split()

format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,TTB,2023-07-21,1.64


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'TTB'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,541,TTB,SET50 / SETHD / SETTHSI,1.47,1.47,1.09,10.82,0.66,"91,937.42","142,261.07",427.04,1.04,541,2017-07-23 07:25:39.022815,2023-01-31 03:08:01.399619


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "TTB" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,TTB,2023-08-11,1.75,1.76,1.73,"748,352,621",1.75
1,TTB,2023-08-10,1.71,1.72,1.69,"285,066,852",1.70
2,TTB,2023-08-09,1.71,1.71,1.68,"285,771,039",1.69
3,TTB,2023-08-08,1.69,1.71,1.68,"278,365,054",1.71
4,TTB,2023-08-07,1.71,1.71,1.68,"255,633,763",1.68


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (yesterday, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2023-08-11' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TTB', 'WHAIR', 'WHART')
ORDER BY name, date


30

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (week_ago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2023-08-03' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTG', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'TTB', 'WHAIR', 'WHART') 
ORDER BY name


30

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(30, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,AH,33.00,31.75,3.94
1,ASK,24.10,23.60,2.12
2,ASP,2.88,2.86,0.70
3,AWC,3.92,4.24,-7.55
4,BCH,18.50,18.10,2.21
5,CPNREIT,11.10,10.90,1.83
6,DIF,9.95,10.10,-1.49
7,GVREIT,7.45,8.05,-7.45
8,ICHI,14.90,14.40,3.47
9,IVL,33.00,33.75,-2.22


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
23,STA,13.90,15.90,-12.58%,Worse
3,AWC,3.92,4.24,-7.55%,Worse
7,GVREIT,7.45,8.05,-7.45%,Worse
14,MCS,6.15,6.45,-4.65%,Worse
16,ORI,10.00,10.30,-2.91%,Worse
24,SYNEX,11.20,11.50,-2.61%,Worse
9,IVL,33.00,33.75,-2.22%,Worse
20,SCC,317.00,324.00,-2.16%,Worse
17,PTG,10.20,10.40,-1.92%,Worse
6,DIF,9.95,10.10,-1.49%,Worse


In [16]:
file_name = 'trend.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name

trend.sort_values(['percent'],ascending=[True]).to_csv(output_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(data_file, index=False)
trend.sort_values(['percent'],ascending=[True]).to_csv(osd_file, index=False)

### Filter only performance = "Worse"

In [17]:
mask = trend.perform == 'Worse'
trend[mask]

,name,price_d,price_w,percent,perform
3,AWC,3.92,4.24,-7.55,Worse
6,DIF,9.95,10.10,-1.49,Worse
7,GVREIT,7.45,8.05,-7.45,Worse
9,IVL,33.00,33.75,-2.22,Worse
14,MCS,6.15,6.45,-4.65,Worse
16,ORI,10.00,10.30,-2.91,Worse
17,PTG,10.20,10.40,-1.92,Worse
20,SCC,317.00,324.00,-2.16,Worse
23,STA,13.90,15.90,-12.58,Worse
24,SYNEX,11.20,11.50,-2.61,Worse


In [18]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,46.67%
Worse,43.33%
No Change,10.00%


In [19]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(251, 8)

In [20]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [21]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(30, 13)

In [22]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
3,AWC,4.24,3.92,-7.55%,Worse,SET50
9,IVL,33.75,33.00,-2.22%,Worse,SET50
20,SCC,324.00,317.00,-2.16%,Worse,SET50
18,PTTGC,39.00,39.50,1.28%,Better,SET50
27,TTB,1.65,1.75,6.06%,Better,SET50
11,JMART,16.40,18.10,10.37%,Better,SET50
12,JMT,37.50,43.25,15.33%,Better,SET50


In [23]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,57.14%
Worse,42.86%


In [24]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
23,STA,15.90,13.90,-12.58%,Worse,SET100
16,ORI,10.30,10.00,-2.91%,Worse,SET100
17,PTG,10.40,10.20,-1.92%,Worse,SET100
19,RCL,24.20,24.20,0.00%,No Change,SET100
4,BCH,18.10,18.50,2.21%,Better,SET100
22,SINGER,7.80,8.60,10.26%,Better,SET100
13,KCE,41.75,46.75,11.98%,Better,SET100


In [25]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,42.86%
Worse,42.86%
No Change,14.29%


In [26]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt
7,GVREIT,8.05,7.45,-7.45%,Worse,SET999
14,MCS,6.45,6.15,-4.65%,Worse,SET999
24,SYNEX,11.50,11.20,-2.61%,Worse,SET999
6,DIF,10.10,9.95,-1.49%,Worse,SET999
29,WHART,10.70,10.60,-0.93%,Worse,SET999
28,WHAIR,6.95,6.90,-0.72%,Worse,SET999
25,TFFIF,7.55,7.50,-0.66%,Worse,SET999
21,SENA,2.98,2.98,0.00%,No Change,SET999
26,TMT,7.20,7.20,0.00%,No Change,SET999
2,ASP,2.86,2.88,0.70%,Better,SET999


In [27]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,43.75%
Worse,43.75%
No Change,12.50%


In [28]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_w,price_d,percent,perform,mrkt


In [29]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
